<a href="https://colab.research.google.com/github/kavyajeetbora/monitoring_water_surface_area/blob/master/notebooks/historical_surface_water.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Monitoring Inland Surface Water Area

In [1]:
!pip install -q rioxarray

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 17.8 MB/s eta 0:00:00


In [2]:
import geemap
import ee
import matplotlib.pyplot as plt
import pandas as pd
import subprocess
from IPython.display import Image as IpyImage
from PIL import Image, ImageFont, ImageDraw, ImageSequence
import rioxarray as rxr
import scipy as sp
import numpy as np
import plotly.graph_objects as go

ee.Authenticate()
ee.Initialize(project='kavyajeetbora-ee')

## Area of Interest

Used this tool to create a geojson file of the area of interest:

[keene Polyline Tool](https://www.keene.edu/campus/maps/tool/?coordinates=77.1200409%2C%2011.5324541%0A76.9923248%2C%2011.5062217%0A76.9916382%2C%2011.3467571%0A77.1529998%2C%2011.4261642%0A77.1200409%2C%2011.5324541) : Use this tool to create a polygon and generate a GeoJson text for further use

In [3]:
geojson  = {
  "coordinates": [[[73.777309817,15.6646429326],[73.9436840308,15.6646429326],[73.9436840308,15.7947142468],[73.777309817,15.7947142468],[73.777309817,15.6646429326]]],
  "type": "Polygon"
}

geometry = ee.Geometry(geojson)

## Define Functions

In [149]:
def maskS2clouds(image):
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask).divide(10000)

def addNDWI(image):
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
    water_mask = ndwi.gte(0).rename('water_mask')
    return image.addBands(ndwi).addBands(water_mask)

## Downloading the Global Surface Water data


The `maskS2clouds` function ensures that only clear-sky pixels are retained in the image, which is crucial for accurate analysis and visualization

In [157]:
# Define the date range for the image collection.
year = 2023
start_date = f'{year}-01-01'
end_date = f'{year}-01-31'

# Load the Sentinel-2 image collection.
dataset = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
            .filterDate(start_date, end_date) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
            .map(maskS2clouds) \
            .map(addNDWI) \
            .filter(ee.Filter.bounds(geometry))

In [155]:
dataset.size()

In [166]:
Map = geemap.Map()

water_image = dataset.median().select('water_mask').selfMask()

vizParams = {
    'max': 1,
    'palette': ['#0561f5']
}

# Define visualization parameters for RGB
rgb_vis_params = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 0.3,
}

# Add the Sentinel-2 image to the map
Map.addLayer(dataset.first(), rgb_vis_params, 'Sentinel-2 RGB')

# Add the bounding box to the map with an outline and no fill
Map.add_basemap("HYBRID")
Map.addLayer(geometry, {'color': 'blue', "fill": False, 'opacity': 0.2}, 'Bounding Box')
Map.addLayer(water_image.clip(geometry), vizParams, 'NDWI')
Map.centerObject(geometry, 12)

Map

Map(center=[15.729687417562605, 73.86049692390029], controls=(WidgetControl(options=['position', 'transparent_…